# My Best 18 - IPL Auction 2026

## Solved using Gurobi

In [1]:
# Run this in a code cell to install the required libraries
!pip install selenium webdriver-manager pandas beautifulsoup4

In [2]:
!pip install tabulate

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [4]:
# --- 1. Robust Selenium Setup ---
chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")

chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("Initializing browser...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.iplt20.com/auction"

try:
    print(f"Accessing {url}...")
    driver.get(url)
    
    wait = WebDriverWait(driver, 20)

    print("Attempting to switch to 'Sold Players' tab...")
    try:
        sold_tab = wait.until(EC.presence_of_element_located((By.ID, "Sold_Players_click")))
        driver.execute_script("arguments[0].click();", sold_tab)
        print("Tab clicked via JavaScript.")
        print("Waiting for player tables to render...")
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sold-players")))
        time.sleep(5)
        
    except Exception as e:
        print(f"Navigation warning: {e}")

        
    print("Parsing HTML...")
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    all_sold_players = []
    tables = soup.find_all('table', class_='sold-players')
    
    print(f"Found {len(tables)} tables containing sold player data.")

    for table in tables:
        section = table.find_parent('section')
        team_name = "Unknown Team"
        
        if section:
            header = section.find('h2')
            if header:
                team_name = header.get_text(strip=True)
        
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            
 
            if len(cols) >= 4:                              
                player_name = cols[1].get_text(strip=True)
                base_price = cols[2].get_text(strip=True)
                winning_bid = cols[3].get_text(strip=True)
                status = cols[4].get_text(strip=True) if len(cols) > 4 else "Sold"

                all_sold_players.append({
                    "Team": team_name,
                    "Player": player_name,
                    "Base_Price": base_price,
                    "Winning_Bid": winning_bid,
                    "Status": status
                })

    if all_sold_players:
        df = pd.DataFrame(all_sold_players)
        print("\n--- Success! Data Sample: ---")
        print(df.head())
        print(f"\nTotal Players Scraped: {len(df)}")
        df.to_csv("ipl_sold_players_2026.csv", index=False)
        print("Data saved to 'ipl_sold_players_2026.csv'")
    else:
        print("No player data found. The page layout might be completely different or blocked.")

except Exception as e:
    print(f"Critical Error: {e}")

finally:
    driver.quit()

Initializing browser...
Accessing https://www.iplt20.com/auction...
Attempting to switch to 'Sold Players' tab...
Tab clicked via JavaScript.
Waiting for player tables to render...
Parsing HTML...
Found 10 tables containing sold player data.

--- Success! Data Sample: ---
                  Team         Player    Base_Price    Winning_Bid    Status
0  Chennai Super Kings  Kartik Sharma    ₹30,00,000  ₹14,20,00,000  Uncapped
1  Chennai Super Kings  Prashant Veer    ₹30,00,000  ₹14,20,00,000  Uncapped
2  Chennai Super Kings   Rahul Chahar  ₹1,00,00,000   ₹5,20,00,000    Capped
3  Chennai Super Kings     Matt Henry  ₹2,00,00,000   ₹2,00,00,000    Capped
4  Chennai Super Kings   Akeal Hosein  ₹2,00,00,000   ₹2,00,00,000    Capped

Total Players Scraped: 77
Data saved to 'ipl_sold_players_2026.csv'


In [5]:
df.head()

,Team,Player,Base_Price,Winning_Bid,Status
0,Chennai Super Kings,Kartik Sharma,"₹30,00,000","₹14,20,00,000",Uncapped
1,Chennai Super Kings,Prashant Veer,"₹30,00,000","₹14,20,00,000",Uncapped
2,Chennai Super Kings,Rahul Chahar,"₹1,00,00,000","₹5,20,00,000",Capped
3,Chennai Super Kings,Matt Henry,"₹2,00,00,000","₹2,00,00,000",Capped
4,Chennai Super Kings,Akeal Hosein,"₹2,00,00,000","₹2,00,00,000",Capped


In [6]:
df["Player"]

0     Kartik Sharma
1     Prashant Veer
2      Rahul Chahar
3        Matt Henry
4      Akeal Hosein
          ...      
72     Praful Hinge
73       Amit Kumar
74    Onkar Tarmale
75    Sakib Hussain
76    Shivang Kumar
Name: Player, Length: 77, dtype: object

In [7]:
df["Captaincy_ratings"] = [
    5.8,  # Kartik Sharma
    5.5,  # Prashant Veer
    7.0,  # Rahul Chahar
    6.5,  # Matt Henry
    6.8,  # Akeal Hosein
    7.5,  # Matthew Short
    6.3,  # Zak Foulkes
    7.8,  # Sarfaraz Khan
    6.0,  # Aman Khan
    5.5,  # Auqib Dar
    7.4,  # Pathum Nissanka
    7.4,  # Kyle Jamieson
    6.5,  # Lungisani Ngidi
    7.6,  # Ben Duckett
    9.0,  # David Miller
    7.5,  # Prithvi Shaw
    5.0,  # Sahil Parakh
    9.5,  # Jason Holder
    6.5,  # Tom Banton
    5.4,  # Ashok Sharma
    6.4,  # Luke Wood
    5.4,  # Prithviraj Yarra
    8.5,  # Cameron Green
    7.2,  # Matheesha Pathirana
    7.2,  # Mustafizur Rahman
    5.4,  # Tejasvi Singh
    8.5,  # Rachin Ravindra
    7.3,  # Finn Allen
    7.3,  # Tim Seifert
    6.8,  # Akash Deep
    7.8,  # Rahul Tripathi
    5.3,  # Daksh Kamra
    5.3,  # Sarthak Ranjan
    6.1,  # Prashant Solanki
    7.0,  # Kartik Tyagi
    8.0,  # Josh Inglis
    6.0,  # Mukul Choudhary
    5.3,  # Akshat Raghuwanshi
    7.2,  # Anrich Nortje
    9.0,  # Wanindu Hasaranga
    5.2,  # Naman Tiwari
    8.8,  # Quinton De Kock
    5.7,  # Mayank Rawat
    6.2,  # Atharva Ankolekar
    5.0,  # Mohammad Izhar
    5.0,  # Danish Malewar
    6.7,  # Ben Dwarshuis
    6.7,  # Cooper Connolly
    5.1,  # Vishal Nishad
    5.0,   # Pravin Dubey
    7.6,  # Ravi Bishnoi
    7.1,  # Adam Milne
    5.7,  # Ravi Singh
    6.0,  # Sushant Mishra
    6.8,  # Kuldeep Sen
    5.0,  # Brijesh Sharma
    5.8,  # Aman Rao Perala
    5.8,  # Vignesh Puthur
    5.0,  # Yash Raj Punja
    8.2,  # Venkatesh Iyer
    5.2,  # Mangesh Yadav
    6.0,  # Jacob Duffy
    6.3,  # Jordan Cox
    5.2,  # Kanishk Chouhan
    5.2,  # Vihaan Malhotra
    6.2,  # Vicky Ostwal
    5.1,  # Satvik Deswal
    8.5,  # Liam Livingstone
    5.7,  # Jack Edwards
    5.0,  # Salil Arora
    6.2,  # Shivam Mavi
    5.0,  # Krains Fuletra
    5.0,  # Praful Hinge
    5.0,  # Amit Kumar
    5.0,  # Onkar Tarmale
    5.0,  # Sakib Hussain
    5.0,  # Shivang Kumar
]

In [8]:
df.head()

,Team,Player,Base_Price,Winning_Bid,Status,Captaincy_ratings
0,Chennai Super Kings,Kartik Sharma,"₹30,00,000","₹14,20,00,000",Uncapped,5.8
1,Chennai Super Kings,Prashant Veer,"₹30,00,000","₹14,20,00,000",Uncapped,5.5
2,Chennai Super Kings,Rahul Chahar,"₹1,00,00,000","₹5,20,00,000",Capped,7.0
3,Chennai Super Kings,Matt Henry,"₹2,00,00,000","₹2,00,00,000",Capped,6.5
4,Chennai Super Kings,Akeal Hosein,"₹2,00,00,000","₹2,00,00,000",Capped,6.8


In [9]:
new_columns = [
    "SR", "Economy", "Total_Match",
    "TopOrder_Batter", "Middle_Order_Batter", "All_Rounder",
    "Pacer", "Spin_Bowler", "WK", "Overseas"
]

data = [
    [162.92, 0, 11, 1, 0, 0, 0, 0, 1, 0],
    [167.16, 6.45, 9, 0, 0, 1, 0, 1, 0, 0],
    [106, 7.68, 140, 0, 0, 0, 0, 1, 0, 0],
    [148.21, 8.22, 174, 0, 0, 0, 1, 0, 0, 1],
    [113.64, 7.08, 263, 0, 0, 0, 0, 1, 0, 1],
    [146.5, 7.8, 150, 1, 0, 0, 0, 0, 0, 1],
    [125.37, 7.94, 63, 0, 1, 1, 1, 0, 0, 1],
    [139.43, 6.75, 103, 0, 1, 0, 0, 0, 0, 0],
    [145.34, 8.8, 32, 0, 1, 0, 1, 0, 0, 0],
    [106.81, 7.74, 34, 0, 0, 0, 0, 0, 0, 0],
    [131.12, 0, 157, 1, 0, 0, 0, 0, 0, 1],
    [127.58, 8.21, 96, 0, 0, 1, 1, 0, 0, 1],
    [79.23, 8.36, 165, 0, 0, 1, 1, 0, 0, 1],
    [140.18, 0, 216, 1, 0, 0, 0, 0, 1, 1],
    [138.31, 10.33, 550, 0, 1, 0, 0, 0, 0, 1],
    [152.04, 0, 124, 1, 0, 0, 0, 0, 0, 0],
    [114.28, 0, 2, 1, 0, 0, 0, 0, 0, 0],
    [135.33, 8.28, 332, 0, 1, 1, 1, 0, 0, 1],
    [142.75, 0, 207, 0, 1, 0, 0, 0, 1, 1],
    [194.11, 9.25, 10, 0, 0, 1, 1, 0, 0, 0],
    [113.28, 8.45, 207, 0, 0, 0, 1, 0, 0, 1],
    [90.9, 8.89, 14, 0, 0, 0, 1, 0, 0, 0],
    [151.07, 9.05, 63, 0, 1, 1, 1, 0, 0, 1],
    [57.35, 8.63, 103, 0, 0, 0, 1, 0, 0, 1],
    [73.7, 7.42, 322, 0, 0, 0, 1, 0, 0, 1],
    [100, 0, 2, 0, 1, 0, 0, 0, 1, 0],
    [139.57, 7.33, 119, 1, 0, 1, 0, 1, 0, 1],
    [174, 8, 171, 1, 0, 0, 0, 0, 1, 1],
    [135.14, 0, 303, 0, 1, 0, 0, 0, 1, 1],
    [186.41, 8.32, 53, 0, 0, 0, 1, 0, 0, 0],
    [134.47, 7.5, 172, 1, 0, 0, 0, 0, 0, 0],
    [100, 10, 2, 0, 1, 1, 0, 1, 0, 0],
    [126.9, 0, 5, 1, 0, 0, 0, 0, 0, 0],
    [50, 8.12, 23, 0, 0, 0, 0, 1, 0, 0],
    [82.35, 9.23, 36, 0, 0, 0, 1, 0, 0, 0],
    [149.56, 0, 166, 0, 1, 0, 0, 0, 1, 1],
    [165.35, 0, 7, 0, 1, 0, 0, 0, 1, 0],
    [131.25, 0, 7, 0, 1, 0, 0, 1, 0, 0],
    [105.88, 7.8, 164, 0, 0, 0, 1, 0, 0, 1],
    [142.56, 6.98, 240, 0, 1, 1, 0, 1, 0, 1],
    [100, 10, 2, 0, 0, 0, 1, 0, 0, 0],
    [138.58, 0, 426, 1, 0, 0, 0, 0, 1, 1],
    [122.58, 7.59, 11, 0, 0, 1, 0, 1, 0, 0],
    [156.97, 7.23, 25, 0, 1, 1, 0, 1, 0, 0],
    [64.28, 7.11, 5, 0, 0, 0, 1, 0, 0, 0],
    [80, 0, 13, 1, 0, 0, 0, 0, 0, 0],
    [141.05, 8.32, 184, 0, 0, 0, 1, 0, 0, 1],
    [134.43, 7.24, 66, 0, 0, 1, 0, 1, 0, 1],
    [40, 10, 2, 0, 0, 0, 0, 1, 0, 0],
    [107.98, 7.25, 33, 0, 0, 0, 0, 1, 0, 0],
    [91.9, 7.54, 166, 0, 0, 0, 0, 1, 0, 0],
    [115.33, 7.89, 226, 0, 0, 0, 1, 0, 0, 1],
    [174.55, 0, 10, 0, 1, 0, 0, 0, 1, 0],
    [30, 9.05, 15, 0, 0, 0, 1, 0, 0, 0],
    [15, 8.93, 39, 0, 0, 0, 1, 0, 0, 0],
    [30, 10, 2, 0, 0, 0, 1, 0, 0, 0],
    [162.7, 0, 11, 1, 0, 0, 0, 0, 0, 0],
    [10, 8.96, 9, 0, 0, 0, 0, 1, 0, 0],
    [10, 10, 2, 0, 0, 0, 1, 0, 0, 0],
    [139.08, 7.76, 145, 0, 0, 0, 1, 0, 0, 0],
    [233, 12.14, 2, 0, 1, 1, 1, 0, 0, 0],
    [129.23, 8.36, 158, 0, 0, 0, 1, 0, 0, 1],
    [140, 0, 173, 0, 1, 0, 0, 0, 1, 1],
    [80, 12, 3, 0, 1, 1, 0, 1, 0, 0],
    [100, 11, 4, 0, 1, 1, 0, 1, 0, 0],
    [106.34, 6.46, 15, 0, 0, 0, 0, 1, 0, 0],
    [30, 9, 2, 0, 0, 0, 0, 1, 0, 0],
    [144.7, 8.48, 340, 1, 1, 1, 0, 1, 0, 1],
    [122.05, 8.21, 74, 0, 1, 1, 1, 0, 0, 1],
    [197.25, 0, 9, 0, 1, 0, 0, 0, 1, 0],
    [114.7, 8.27, 68, 0, 1, 1, 1, 0, 0, 0],
    [0, 10.05, 2, 0, 0, 0, 0, 1, 0, 0],
    [0, 5.75, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 8.3, 5, 0, 0, 0, 0, 1, 0, 0],
    [0, 5.61, 4, 0, 0, 0, 1, 0, 0, 0],
    [93.1, 10.34, 12, 0, 0, 0, 1, 0, 0, 0],
    [143.9, 8.05, 4, 0, 0, 0, 0, 1, 0, 0]
]

new_df = pd.DataFrame(data, columns=new_columns)


In [10]:
new_df.head()

,SR,Economy,Total_Match,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas
0,162.92,0.00,11,1,0,0,0,0,1,0
1,167.16,6.45,9,0,0,1,0,1,0,0
2,106.00,7.68,140,0,0,0,0,1,0,0
3,148.21,8.22,174,0,0,0,1,0,0,1
4,113.64,7.08,263,0,0,0,0,1,0,1


In [11]:
final_df = df.join(new_df)
final_df.head()

,Team,Player,Base_Price,Winning_Bid,Status,Captaincy_ratings,SR,Economy,Total_Match,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas
0,Chennai Super Kings,Kartik Sharma,"₹30,00,000","₹14,20,00,000",Uncapped,5.8,162.92,0.00,11,1,0,0,0,0,1,0
1,Chennai Super Kings,Prashant Veer,"₹30,00,000","₹14,20,00,000",Uncapped,5.5,167.16,6.45,9,0,0,1,0,1,0,0
2,Chennai Super Kings,Rahul Chahar,"₹1,00,00,000","₹5,20,00,000",Capped,7.0,106.00,7.68,140,0,0,0,0,1,0,0
3,Chennai Super Kings,Matt Henry,"₹2,00,00,000","₹2,00,00,000",Capped,6.5,148.21,8.22,174,0,0,0,1,0,0,1
4,Chennai Super Kings,Akeal Hosein,"₹2,00,00,000","₹2,00,00,000",Capped,6.8,113.64,7.08,263,0,0,0,0,1,0,1


### Remove Currency Symbol : 

In [12]:
final_df['Base_Price'] = final_df['Base_Price'].str.replace('₹', '').str.replace(',', '').astype(float)
final_df['Winning_Bid'] = final_df['Winning_Bid'].str.replace('₹', '').str.replace(',', '').astype(float)
final_df.head()

,Team,Player,Base_Price,Winning_Bid,Status,Captaincy_ratings,SR,Economy,Total_Match,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas
0,Chennai Super Kings,Kartik Sharma,3000000.0,142000000.0,Uncapped,5.8,162.92,0.00,11,1,0,0,0,0,1,0
1,Chennai Super Kings,Prashant Veer,3000000.0,142000000.0,Uncapped,5.5,167.16,6.45,9,0,0,1,0,1,0,0
2,Chennai Super Kings,Rahul Chahar,10000000.0,52000000.0,Capped,7.0,106.00,7.68,140,0,0,0,0,1,0,0
3,Chennai Super Kings,Matt Henry,20000000.0,20000000.0,Capped,6.5,148.21,8.22,174,0,0,0,1,0,0,1
4,Chennai Super Kings,Akeal Hosein,20000000.0,20000000.0,Capped,6.8,113.64,7.08,263,0,0,0,0,1,0,1


### Weighted T20 Impact Score

We will calculate a **`Impact_Score`** using the following logic. This normalizes everything to a roughly 0-100 scale.

$$
\text{Score} = (\text{Batting}_{pts}) + (\text{Bowling}_{pts}) + (\text{Experience}_{pts}) + (\text{Captaincy}_{pts})
$$

#### 1. The Components

- **Batting Points (Aggression):**
    * Higher Strike Rate (SR) is better.
    * Formula: $\text{SR} \times 0.4$
    * Only apply if the player is a Top Order, Middle Order, Wicket Keeper, or All-Rounder.

- **Bowling Points (Control):**
    * Lower Economy is better. T20 par economy is roughly 8.5. We reward players below this and penalize those above.
    * $(14.0 - \text{Economy}) \times 10$
    *  Only apply if the player is a Pacer, Spinner, or All-Rounder. (If Economy is 0, treat as neutral/0).

- **Experience Points (Reliability):**
    * More matches mean the stats are more reliable.
    * $\text{Total\_Match} \times 0.5$

- **Captaincy Points (Leadership):**
    * A rating of 5 is average. We reward high ratings.
    * $(\text{Captaincy\_ratings} - 5) \times 10$

In [13]:
import numpy as np

condition1 = (
    (final_df['TopOrder_Batter'] == 1) | 
    (final_df['Middle_Order_Batter'] == 1) | 
    (final_df['All_Rounder'] == 1) | 
    (final_df['WK'] == 1)
)

final_df['Batting_Calc'] = np.where(condition1, final_df['SR'] * 0.4, 0)

In [14]:
condition2 = (
    (final_df['Pacer'] == 1) | 
    (final_df['Spin_Bowler'] == 1) | 
    (final_df['All_Rounder'] == 1)
) & (final_df['Economy'] > 0)

final_df['Bowling_Calc'] = np.where(condition2, (14.0 - final_df['Economy']) * 10, 0)
final_df.head()

,Team,Player,Base_Price,Winning_Bid,Status,Captaincy_ratings,SR,Economy,Total_Match,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas,Batting_Calc,Bowling_Calc
0,Chennai Super Kings,Kartik Sharma,3000000.0,142000000.0,Uncapped,5.8,162.92,0.00,11,1,0,0,0,0,1,0,65.168,0.0
1,Chennai Super Kings,Prashant Veer,3000000.0,142000000.0,Uncapped,5.5,167.16,6.45,9,0,0,1,0,1,0,0,66.864,75.5
2,Chennai Super Kings,Rahul Chahar,10000000.0,52000000.0,Capped,7.0,106.00,7.68,140,0,0,0,0,1,0,0,0.000,63.2
3,Chennai Super Kings,Matt Henry,20000000.0,20000000.0,Capped,6.5,148.21,8.22,174,0,0,0,1,0,0,1,0.000,57.8
4,Chennai Super Kings,Akeal Hosein,20000000.0,20000000.0,Capped,6.8,113.64,7.08,263,0,0,0,0,1,0,1,0.000,69.2


In [15]:
final_df['Exp_Calc'] = final_df['Total_Match'] * 0.5
final_df['Cap_Calc'] = (final_df['Captaincy_ratings'] - 5) * 10

final_df['Performance_Score'] = (
    final_df['Batting_Calc'].fillna(0) + 
    final_df['Bowling_Calc'].fillna(0) + 
    final_df['Exp_Calc'].fillna(0) + 
    final_df['Cap_Calc'].fillna(0)
)

final_df['Value_Index'] = final_df['Performance_Score'] / (final_df['Winning_Bid'] / 10000000)

print(final_df[['Team', 'Player', 'Performance_Score', 'Winning_Bid']]
      .sort_values(by='Performance_Score', ascending=False)
      .head(10))

                     Team             Player  Performance_Score  Winning_Bid
14         Delhi Capitals       David Miller            370.324   20000000.0
17         Gujarat Titans       Jason Holder            322.332   70000000.0
67    Sunrisers Hyderabad   Liam Livingstone            318.080  130000000.0
41         Mumbai Indians    Quinton De Kock            306.432   10000000.0
39   Lucknow Super Giants  Wanindu Hasaranga            287.224   20000000.0
24  Kolkata Knight Riders  Mustafizur Rahman            248.800   92000000.0
28  Kolkata Knight Riders        Tim Seifert            228.556   15000000.0
4     Chennai Super Kings       Akeal Hosein            218.700   20000000.0
26  Kolkata Knight Riders    Rachin Ravindra            217.028   20000000.0
51       Rajasthan Royals         Adam Milne            195.100   24000000.0


In [16]:
final_df['Required_Bid(Lakh)'] = final_df['Winning_Bid']/1000000
final_df.head()

,Team,Player,Base_Price,Winning_Bid,Status,Captaincy_ratings,SR,Economy,Total_Match,TopOrder_Batter,...,Spin_Bowler,WK,Overseas,Batting_Calc,Bowling_Calc,Exp_Calc,Cap_Calc,Performance_Score,Value_Index,Required_Bid(Lakh)
0,Chennai Super Kings,Kartik Sharma,3000000.0,142000000.0,Uncapped,5.8,162.92,0.00,11,1,...,0,1,0,65.168,0.0,5.5,8.0,78.668,5.540000,142.0
1,Chennai Super Kings,Prashant Veer,3000000.0,142000000.0,Uncapped,5.5,167.16,6.45,9,0,...,1,0,0,66.864,75.5,4.5,5.0,151.864,10.694648,142.0
2,Chennai Super Kings,Rahul Chahar,10000000.0,52000000.0,Capped,7.0,106.00,7.68,140,0,...,1,0,0,0.000,63.2,70.0,20.0,153.200,29.461538,52.0
3,Chennai Super Kings,Matt Henry,20000000.0,20000000.0,Capped,6.5,148.21,8.22,174,0,...,0,0,1,0.000,57.8,87.0,15.0,159.800,79.900000,20.0
4,Chennai Super Kings,Akeal Hosein,20000000.0,20000000.0,Capped,6.8,113.64,7.08,263,0,...,1,0,1,0.000,69.2,131.5,18.0,218.700,109.350000,20.0


In [17]:
decider_df_cols = ["Team","Player","Captaincy_ratings","SR","Economy","TopOrder_Batter",
    "Middle_Order_Batter",    "All_Rounder","Pacer","Spin_Bowler","WK","Overseas",
    "Performance_Score","Required_Bid(Lakh)" ]
decider_df = final_df[decider_df_cols]
decider_df.head()

,Team,Player,Captaincy_ratings,SR,Economy,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas,Performance_Score,Required_Bid(Lakh)
0,Chennai Super Kings,Kartik Sharma,5.8,162.92,0.00,1,0,0,0,0,1,0,78.668,142.0
1,Chennai Super Kings,Prashant Veer,5.5,167.16,6.45,0,0,1,0,1,0,0,151.864,142.0
2,Chennai Super Kings,Rahul Chahar,7.0,106.00,7.68,0,0,0,0,1,0,0,153.200,52.0
3,Chennai Super Kings,Matt Henry,6.5,148.21,8.22,0,0,0,1,0,0,1,159.800,20.0
4,Chennai Super Kings,Akeal Hosein,6.8,113.64,7.08,0,0,0,0,1,0,1,218.700,20.0


In [18]:
team_short_names = {
    'Chennai Super Kings': 'CSK',
    'Delhi Capitals': 'DC',
    'Gujarat Titans': 'GT',
    'Kolkata Knight Riders': 'KKR',
    'Lucknow Super Giants': 'LSG',
    'Mumbai Indians': 'MI',
    'Punjab Kings': 'PBKS',
    'Rajasthan Royals': 'RR',
    'Royal Challengers Bengaluru': 'RCB',
    'Sunrisers Hyderabad': 'SRH'
}

decider_df['Team'] = decider_df['Team'].replace(team_short_names)

decider_df.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1580\1070832562.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decider_df['Team'] = decider_df['Team'].replace(team_short_names)


,Team,Player,Captaincy_ratings,SR,Economy,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas,Performance_Score,Required_Bid(Lakh)
0,CSK,Kartik Sharma,5.8,162.92,0.00,1,0,0,0,0,1,0,78.668,142.0
1,CSK,Prashant Veer,5.5,167.16,6.45,0,0,1,0,1,0,0,151.864,142.0
2,CSK,Rahul Chahar,7.0,106.00,7.68,0,0,0,0,1,0,0,153.200,52.0
3,CSK,Matt Henry,6.5,148.21,8.22,0,0,0,1,0,0,1,159.800,20.0
4,CSK,Akeal Hosein,6.8,113.64,7.08,0,0,0,0,1,0,1,218.700,20.0


- $N$ is the number of players sold in IPL Auction 2026
  
- $n$ is set of attributes considered for selecting players, with $n=\{1,2,...,n\}$

- $T$ is set of numbers that we need to take to fulfil our player requirements
- $A$ is Matrix of binary values that defines players profile

- $P$ Performance Score matrix, with $p_i$ values
- $c_{i}$ is bid amount that spent to buy $i^{th}$ Player, in C Matrix
- $x_i$ is the binary vector, $x_j \in ${$1,0$}

Then, the formulation is the following
$$\begin{align}
\max \quad & \sum_{i\in N} p_{i} x_{i} \\
\text{s.t.} \quad & \sum_{i\in N} c_i x_{i} <= MaxBid  && i \in N \\
& \sum_{j\in n}\sum_{i\in N} a_{ij}x_{i} >= t_j && j \in n \\
& a_{ij} \in \{0,1\} \\
& x_{i} \in \{0,1\} \\
& p_{i} \in \mathbb{R}^{+} \\
& c_{i} \in [0,300]\\
\end{align}$$

In [19]:
import gurobipy as gp
from gurobipy import GRB

BUDGET_LIMIT = 20000 
TEAM_SIZE = 18

m = gp.Model("DreamTeam_Selection")

players = decider_df.index
x = m.addVars(players, vtype=GRB.BINARY, name="Player")


m.setObjective(
    gp.quicksum(x[i] * decider_df.loc[i, "Performance_Score"] for i in players), 
    GRB.MAXIMIZE
)


m.addConstr(gp.quicksum(x[i] for i in players) == TEAM_SIZE, "TeamSize")


m.addConstr(
    gp.quicksum(x[i] * decider_df.loc[i, "Required_Bid(Lakh)"] for i in players) <= BUDGET_LIMIT, 
    "Budget"
)


m.addConstr(
    gp.quicksum(x[i] * decider_df.loc[i, "TopOrder_Batter"] for i in players) >= 3, 
    "OverseasLimit"
)
m.addConstr(
    gp.quicksum(x[i] * decider_df.loc[i, "Middle_Order_Batter"] for i in players) >= 3, 
    "OverseasLimit"
)
m.addConstr(
    gp.quicksum(x[i] * decider_df.loc[i, "Overseas"] for i in players) <= 5, 
    "OverseasLimit"
)


m.addConstr(
    gp.quicksum(x[i] * decider_df.loc[i, "WK"] for i in players) >= 2, 
    "Min_WK"
)


m.addConstr(
    gp.quicksum(x[i] * (decider_df.loc[i, "Pacer"]) for i in players) >= 4, 
    "Min_PBowlers"
)
m.addConstr(
    gp.quicksum(x[i] * (decider_df.loc[i, "Spin_Bowler"]) for i in players) >= 2, 
    "Min_SBowlers"
)


m.addConstr(
    gp.quicksum(x[i] * decider_df.loc[i, "All_Rounder"] for i in players) >= 3, 
    "Min_AR"
)

m.optimize()


if m.status == GRB.OPTIMAL:
    selected_indices = [i for i in players if x[i].X > 0.5]  #x[i].X > 0.5 is because of floating point error issue. 1 may be stored as 0.99999..
    best_team = decider_df.loc[selected_indices]
    
    print(f"\nOptimization Successful! Total Score: {m.ObjVal:.2f}")
    print(f"Total Cost: ₹{best_team["Required_Bid(Lakh)"].sum()} Lakhs")
    print("-" * 60)
    print(best_team[["Player", "Performance_Score", "Required_Bid(Lakh)","TopOrder_Batter","Middle_Order_Batter", "WK","All_Rounder", "Spin_Bowler", "Pacer", "Team","Overseas"]])
else:
    print("No feasible solution found. Try increasing budget or relaxing constraints.")

Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 9 rows, 77 columns and 306 nonzeros (Max)
Model fingerprint: 0x57ff375f
Model has 77 linear objective coefficients
Variable types: 0 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Found heuristic solution: objective 1871.7920000
Presolve removed 0 rows and 3 columns
Presolve time: 0.02s
Presolved: 9 rows, 74 columns, 212 nonzeros
Found heuristic solution: objective 3484.3120000
Variable types: 0 continuous, 74 integer (72 binary)

Root relaxation: cutoff, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes

In [20]:
best18 = best_team.sort_values(by=["TopOrder_Batter","Middle_Order_Batter", "WK","All_Rounder","Performance_Score"], ascending=[False,False,False,False,False])
best18 = best18.reset_index(drop=True)
best18.index = best18.index +1
best18

,Team,Player,Captaincy_ratings,SR,Economy,TopOrder_Batter,Middle_Order_Batter,All_Rounder,Pacer,Spin_Bowler,WK,Overseas,Performance_Score,Required_Bid(Lakh)
1,SRH,Liam Livingstone,8.5,144.70,8.48,1,1,1,0,1,0,1,318.080,130.0
2,MI,Quinton De Kock,8.8,138.58,0.00,1,0,0,0,0,1,1,306.432,10.0
3,KKR,Rahul Tripathi,7.8,134.47,7.50,1,0,0,0,0,0,0,167.788,7.5
4,DC,Prithvi Shaw,7.5,152.04,0.00,1,0,0,0,0,0,0,147.816,7.5
5,SRH,Salil Arora,5.0,197.25,0.00,0,1,0,0,0,1,0,83.400,15.0
6,GT,Jason Holder,9.5,135.33,8.28,0,1,1,1,0,0,1,322.332,70.0
7,LSG,Wanindu Hasaranga,9.0,142.56,6.98,0,1,1,0,1,0,1,287.224,20.0
8,MI,Atharva Ankolekar,6.2,156.97,7.23,0,1,1,0,1,0,0,154.988,3.0
9,SRH,Shivam Mavi,6.2,114.70,8.27,0,1,1,1,0,0,0,149.180,7.5
10,DC,David Miller,9.0,138.31,10.33,0,1,0,0,0,0,1,370.324,20.0
